In [86]:
# Answers
# 1. 19
# 2. 42.59
# 3. 98
# 4. 515
# 5. 7.649
# 6. 

In [72]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [ ]:
file1 = '/Users/artem/Dev/mlops-course/data/yellow_tripdata_2023-01.parquet'
file2 = '/Users/artem/Dev/mlops-course/data/yellow_tripdata_2023-02.parquet'

In [16]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    # df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # df.duration = df.duration.apply(lambda x : x.total_seconds() / 60)
    # df = df[(df.duration >= 1)&(df.duration <= 60)]
    # categorical = ['PULocationID', 'DOLocationID']
    # df[categorical] = df[categorical].astype(str)
    return df

In [17]:
df_jan = read_dataframe(file1)
df_feb = read_dataframe(file2)

In [27]:
df_jan.shape

(3066766, 19)

In [28]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [29]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda x : x.total_seconds() / 60)

In [39]:
df_jan.duration.std()

np.float64(42.59435124195458)

In [40]:
df_jan.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [41]:
df_jan = df_jan[(df_jan.duration >= 1)&(df_jan.duration <= 60)]

In [43]:
df_jan.shape

(3009173, 20)

In [45]:
3009173 / 3066766 * 100

98.1220282212598

In [52]:
categorical = ['PULocationID', 'DOLocationID']
df_jan[categorical] = df_jan[categorical].astype(str)
df_jan['PU_DO'] = df_jan['PULocationID'] + '_' + df_jan['PULocationID']

/var/folders/19/vccrcyks6xx69t_4kdjm61mh0000gn/T/ipykernel_9174/2189571335.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan[categorical] = df_jan[categorical].astype(str)
/var/folders/19/vccrcyks6xx69t_4kdjm61mh0000gn/T/ipykernel_9174/2189571335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan['PU_DO'] = df_jan['PULocationID'] + '_' + df_jan['PULocationID']


In [164]:
dv = DictVectorizer()

df_jan_dict = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')
df_jan_train = dv.fit_transform(df_jan_dict)

In [165]:
lr = LinearRegression()

In [166]:
target = 'duration'
y_train = df_jan[target].values

In [167]:
lr.fit(df_jan_train, y_train)

LinearRegression()

In [168]:
# y_pred = lr.predict(dv.transform(df_jan_dict))
y_pred = lr.predict(df_jan_train)

root_mean_squared_error(y_train, y_pred)

7.6492622258678935

In [169]:
df_feb['duration'] = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime
df_feb.duration = df_feb.duration.apply(lambda x : x.total_seconds() / 60)
df_feb[['PULocationID', 'DOLocationID']] = df_feb[['PULocationID', 'DOLocationID']].astype(str)

In [180]:
df_feb

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1.0,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142.0,163.0,2.0,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
1,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2.0,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71.0,71.0,4.0,3.00,1.00,0.5,0.00,0.0,1.0,5.50,0.0,0.00,0.233333
3,1.0,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132.0,26.0,1.0,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2.0,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161.0,145.0,1.0,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2.0,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249.0,140.0,0.0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,19.000000
2913951,2.0,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186.0,79.0,0.0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,11.133333
2913952,2.0,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158.0,143.0,0.0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,14.000000
2913953,2.0,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79.0,162.0,0.0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,7.000000


In [171]:
df_feb_dict = df_feb[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [172]:
df_feb_dict[:10]

[{'PULocationID': '142.0', 'DOLocationID': '163.0'},
 {'PULocationID': '71.0', 'DOLocationID': '71.0'},
 {'PULocationID': '71.0', 'DOLocationID': '71.0'},
 {'PULocationID': '132.0', 'DOLocationID': '26.0'},
 {'PULocationID': '161.0', 'DOLocationID': '145.0'},
 {'PULocationID': '148.0', 'DOLocationID': '236.0'},
 {'PULocationID': '137.0', 'DOLocationID': '244.0'},
 {'PULocationID': '263.0', 'DOLocationID': '141.0'},
 {'PULocationID': '48.0', 'DOLocationID': '243.0'},
 {'PULocationID': '114.0', 'DOLocationID': '211.0'}]

In [173]:
df_feb_v = dv.transform(df_feb_dict)

In [174]:
df_feb_true = df_feb[target].values

In [175]:
df_feb_v.shape

(2913955, 515)

In [176]:
feb_pred = lr.predict(df_feb_v)

In [179]:
root_mean_squared_error(feb_pred, df_feb_true)

43.440655690582126